### Tutorial Notebook Showing How sus, infoenginessims work together to make a sim object

In [ ]:
#standard imports, these can be done automatically using a conda environemnt
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
# imports to deal with getting nice animations and outputs for matplotlip in jupyter
plt.rcParams["animation.html"] = "jshtml"
from IPython.display import HTML
import matplotlib.animation as animation
%matplotlib inline



In [ ]:
# I always keep my code in a folder in my home directory called 'source/', and begin with the following:
source_path = os.path.expanduser('~/source/')
sys.path.append(source_path)
# which will enable the following imports:
from sus import protocol_designer as pd
from sus.library.szilard_protocols import blw_szilard

# usually these would not be imported but handled in a separate document, but to show the inner workings of the simtools package, I will manually import some stuff

sim_path = os.path.dirname(source_path + "simtools/infoenginessims/")
sys.path.append(sim_path)

from integrators import rkdeterm_eulerstoch
from dynamics import langevin_underdamped, langevin_overdamped
from simprocedures import basic_simprocedures as sp
from simprocedures import running_measurements as rp
from simulation import Simulation

In [ ]:
# first, we have our system, as explained in the sus package tutorial:
system = blw_szilard

system.potential?

In [ ]:
# The system we chose is a 2D potential that uses frou wells to implement a Szilard Cycle over 6 substages
system.protocol.show_substage_times()
# Here are smoe snapshots of the potential
system.show_potential(0)
system.show_potential(.16)
system.show_potential(.66)


In [ ]:
%%capture
# and an animation of the whole thing. (Note there are some qorkarounds here ot get animations working in jupyter noteobok. This is one of the weaker parts of the package, but its just for prototyping so I dont worry about it too much).
ani = system.animate_protocol()
HTML(ani.to_jshtml(fps=30))

In [ ]:
ani

In [ ]:
# next, we want to generate a starting distribution. Generally its an EQ dist at a certain time (the starting time, here). We take a sample of 10_000 initial conditions from the eq distribution

init_state = system.eq_state(10_000, 0, beta=1)

In [ ]:
#the phase space is 2X2 because its a 2D potential and we have 10_000 trials so the shape is...
print(init_state.shape)


In [ ]:
#scatter plot briefly can confirm we have the right dist. 
fig, ax = plt.subplots(1,2, figsize=(12,5))
ax[0].scatter(init_state[:,0,0], init_state[:,1,0])
ax[0].set_xlabel('x')
ax[0].set_xlabel('y')

ax[1].scatter(init_state[:,0,1], init_state[:,1,1])
ax[1].set_xlabel('v_x')
ax[1].set_xlabel('v_y')

In [ ]:
#Now, we set up a sim object, to simulate an equation of motion for the system, with the given protocol and time dependent potential

#FIRST, we choose a dynamic. These cam be built, but we have some built in already for langevin dynamics. Let's go for the full underdamped EOM. For more info look inside the docstrings in the simtools package

gamma, theta, eta = 1, 1, 1

#underdamped langevin dynamics takes 3 parameters and the time dependent force function for our system
dynamic = langevin_underdamped.LangevinUnderdamped(theta, gamma, eta, system.get_external_force)
#it also needs an inertial parameter, the mass. By default all systems have an object mass of 1, but it can be changed
dynamic.mass = system.mass

In [ ]:
# SECOND, we need an integrator to integrate the dynamic. Here the integrator is very simple, RK4 for the deterministic part and Euler for the stochastic. This is a place where we could optimize for more intelligent integrators for sure, but they mostly get the job done as is. Again, check the relevant docstrings for more info.

integrator = rkdeterm_eulerstoch.RKDetermEulerStoch(dynamic)

In [ ]:
# Third, we set up procedues to be done. They can happen before the simualtion, throughout it, or at the end. Check relevant docsrings inside of simtools for more info

procedures = [
            sp.ReturnFinalState(),
            sp.MeasureAllState(trial_request=slice(0, 300)),
            rp.MeasureAllValue(rp.get_dW, 'all_W'), 
            rp.MeasureFinalValue(rp.get_dW, 'final_W')]

#These four procedures do exactly what it sounds like they would do. The first will give us just the final state for all trials. The second measures the whole simulation for the first 100 trials. The third measures work over time, and the last gives us the next work for the whole protocol.

In [ ]:
# Finally, we decide what dt we are going to use:
dt = .0005
nsteps = int(system.protocol.t_f/dt)
# and create a Simulation object
sim = Simulation(integrator.update_state, procedures, nsteps, dt, initial_state=init_state)
# and associate the system with it, for future reference
sim.system = system


In [ ]:
#Now this is all set up, we can run a simulation:
sim.output = sim.run(verbose=True)

In [ ]:
sim.output?

In [ ]:
#extract the results for viewing. The sim.output Bunch object acts a lot like a container for dictionaries and numpy arrays that are callable as attributes
all_state = sim.output.all_state['states']
final_state = sim.output.final_state
final_W = sim.output.final_W
all_W = sim.output.all_W


In [ ]:
#all the outputs are numpy arrays
print(all_W.shape,':trials, steps')

print(all_state.shape,':trials, steps, dimensions, attributes per dimension')

In [ ]:
#work over time for the first 500 trials
plt.plot(all_W[:500,:].transpose(), alpha=.2);

In [ ]:
#just net work for all trials:
plt.hist(final_W, bins=50, density=True);

In [ ]:
fig,ax = plt.subplots(3, figsize=(10,10))
#here are x,v and v_x plots over time
ax[0].plot(all_state[:,:,0,0].transpose());
ax[1].plot(all_state[:,:,1,0].transpose());
ax[2].plot(all_state[:,:,0,1].transpose());

In [ ]:
%%capture
#we can even see an animation of the trajectories where we saved every step. Here the slice is [:,:,:,0], which means we are looking at all trials, all timesteps, all dimensions, but only at position degrees of freedom. [:,:,:,1] would look at velocity degrees of freedom.
ani,_,_ = kt.animate_sim(all_state[:,:,:,0], frame_skip=100, color_by_state=True)
HTML(ani.to_jshtml(fps=30))

In [ ]:
ani

From the above plots, it is clear that this protocol is NOT working as intended. The particles are not reacting quickly enough to the potential, so the control and paramters arent good ones. Can you mess with the different parameters of the dynamic and the system to get a protocol that works better? Start just by making the protocol take longer. Let's make it last for four time units instead of one. One way to do this is to use the system.protocol.time_stretch method. Ill guide you below. This should help a little, at least

In [ ]:
# usually its goo to normalize before stretching, so you dont loose track of the timescale from doing multiple stretches
system.protocol.normalize()
system.protocol.time_stretch(4)

In [ ]:
system.protocol.times

In [ ]:
# typically, I dont remake the whole sim item by hand each time, becuase there is alot of steps that are done over and over. It is faster to use something like the setup_sim function from the beginning. Then you get easier to read code like:

from quick_sim import setup_sim

#we wont need to generate a new initial_state, so thats all we need to make a new sim:


sim = setup_sim(system, init_state, sim_params=[1,1,1], dt=.005, damping=2)
#here, the [1,1,1] is for the dynamic parameters gamma, theta and eta

#we also eased up on the dt. A fairly large dt is fine for sims if you dont need accuracy and are just looking at very general behavior. dt needs to be set carefully when looking to get real results though.

In [ ]:
sim.output = sim.run(verbose=True)

In [ ]:
%%capture
#when doing rapid prototyping like this, an animation can be really useful to see the big picture. 

all_state = sim.output.all_state['states']

ani,_,_ = kt.animate_sim(all_state[:,:,:,0], frame_skip=10, color_by_state=True)
HTML(ani.to_jshtml(fps=30))

In [ ]:
ani

See? It is better, but it isnt great. Ideally you would have the Orange/Blue evenly split along the bottom and the Red/Green evenly split along the top. Try some more stuff. Specifically investigate the dynamic parameters gamma, theta, eta and also the energy scale of the potential through system.potential.scale